<a href="https://colab.research.google.com/github/CARRIEE-WU/AIProject/blob/master/cat_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Dogs and Cats Classification**

## Connect to GitHub

In [1]:
!git clone https://github.com/a945120/AIProject.git

Cloning into 'AIProject'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 43 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (43/43), 315.67 KiB | 13.15 MiB/s, done.
Resolving deltas: 100% (9/9), done.


## Import Modules

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import tqdm
from keras.preprocessing.image import load_img
warnings.filterwarnings('ignore')

## Create Dataframe for Input and Output

In [ ]:
input_path = []
label = []

for class_name in os.listdir("PetImages"):
  for path in os.listdir("PetImages/"+class_name):
    if class_name == 'Cat':
      label.append(0)
    else
      label.append(1)
    input_path.append(os.path.join("PetImages", class_name, path))
print(input_path[0], label[0])